In [48]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
from unidecode import unidecode
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [50]:
# ==========================================
# 1. SETUP & DATA LOADING
# ==========================================
print("--- 1. Loading Data ---")
players_fb = pd.read_csv("C:/Users/ASUS/Desktop/player_full.csv")
players_fif_roles = pd.read_csv("C:/Users/ASUS/Documents/merged_players_fc25.csv")
players_fif = pd.read_csv("C:/Users/ASUS/Downloads/new-players-data-full.csv")

# Filter Active Players (2025)
players_fb['is_active'] = players_fb.groupby('Url')['Season_End_Year'].transform(lambda x: (x == 2025).any())
players_fb = players_fb[players_fb['is_active']].copy().reset_index(drop=True)

--- 1. Loading Data ---


In [51]:
# ==========================================
# 2. PREPROCESSING
# ==========================================
print("--- 2. Preprocessing & Cleaning ---")

def clean_string(series):
    return (series.astype(str).apply(unidecode).str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True))

players_fb['clean_name'] = clean_string(players_fb['Player'])
players_fb['clean_club'] = clean_string(players_fb['Team'])
players_fif['clean_name'] = clean_string(players_fif['full_name'])
players_fif['clean_club'] = clean_string(players_fif['club_name'])

# --- ROLE EXTRACTION ---
def extract_clean_roles(role_str):
    if pd.isna(role_str): return None
    # 1. Split by comma
    parts = str(role_str).split(',')
    # 2. Filter for roles containing '+' 
    valid_roles = []
    for r in parts:
        if '+' in r:
            # Remove + and ++ & strip whitespace
            clean_r = r.replace('+', '').strip()
            valid_roles.append(clean_r)
            
    return ", ".join(valid_roles) if valid_roles else None

players_fif_roles['clean_roles'] = players_fif_roles['roles'].apply(extract_clean_roles)

# Merge roles into main FIFA dataframe
merge_key = 'player_id' if 'player_id' in players_fif.columns and 'player_id' in players_fif_roles.columns else 'full_name'
players_fif = players_fif.merge(players_fif_roles[[merge_key, 'clean_roles']], on=merge_key, how='left')

# Standardize Club Names
club_corrections = {
    'manchester utd': 'manchester united', 'nottm forest': 'nottingham forest',
    'sheffield utd': 'sheffield united', 'luton': 'luton town',
    'wolves': 'wolverhampton wanderers', 'brighton': 'brighton hove albion',
    'tottenham': 'tottenham hotspur', 'west ham': 'west ham united',
    'newcastle': 'newcastle united', 'leeds': 'leeds united',
    'leicester': 'leicester city', 'norwich': 'norwich city',
    'psg': 'paris saint germain', 'inter': 'inter milan',
    'milan': 'ac milan', 'sporting cp': 'sporting cp',
    'real betis': 'real betis balompie', 'ath bilbao': 'athletic club de bilbao',
    'atletico madrid': 'atletico de madrid', 'bayer leverkusen': 'bayer 04 leverkusen',
    'borussia mglad': 'borussia monchengladbach', 'bayern munich': 'fc bayern munchen'
}
players_fb['clean_club'] = players_fb['clean_club'].replace(club_corrections)

--- 2. Preprocessing & Cleaning ---


In [52]:
# ==========================================
# 3. MATCHING
# ==========================================
print("--- 3. Matching Players ---")

players_fb['fc26_pos'] = None
players_fb['fc26_foot'] = None 
players_fb['fc26_roles'] = None

fifa_lookup_all = players_fif.drop_duplicates('clean_name').set_index('clean_name')[['positions', 'preferred_foot', 'clean_roles']].to_dict('index')
fifa_all_names = list(fifa_lookup_all.keys())

# 1: Club Match
common_clubs = set(players_fb['clean_club']).intersection(set(players_fif['clean_club']))

for club in common_clubs:
    fb_subset = players_fb[players_fb['clean_club'] == club]
    fif_subset = players_fif[players_fif['clean_club'] == club]
    fifa_lookup_club = fif_subset.drop_duplicates('clean_name').set_index('clean_name')[['positions', 'preferred_foot', 'clean_roles']].to_dict('index')
    fifa_club_names = list(fifa_lookup_club.keys())

    for idx, row in fb_subset.iterrows():
        p_name = row['clean_name']
        match_data = None
        if p_name in fifa_lookup_club:
            match_data = fifa_lookup_club[p_name]
        else:
            match = process.extractOne(p_name, fifa_club_names, scorer=fuzz.token_set_ratio)
            if match and match[1] >= 65:
                match_data = fifa_lookup_club[match[0]]
        
        if match_data:
            players_fb.at[idx, 'fc26_pos'] = match_data['positions']
            players_fb.at[idx, 'fc26_foot'] = match_data['preferred_foot']
            players_fb.at[idx, 'fc26_roles'] = match_data['clean_roles']

# 2: Global Search
missing_mask = (players_fb['fc26_roles'].isna()) | (players_fb['fc26_roles'] == '')
missing_names = players_fb.loc[missing_mask, 'clean_name'].dropna().unique()

updates = {}
for fb_name in missing_names:
    match_data = None
    if fb_name in fifa_lookup_all:
        match_data = fifa_lookup_all[fb_name]
    else:
        match = process.extractOne(fb_name, fifa_all_names, scorer=fuzz.token_set_ratio)
        if match and match[1] >= 90: 
            match_data = fifa_lookup_all[match[0]]
    if match_data:
        updates[fb_name] = match_data

for idx, row in players_fb[missing_mask].iterrows():
    if row['clean_name'] in updates:
        data = updates[row['clean_name']]
        players_fb.at[idx, 'fc26_roles'] = data['clean_roles']
        players_fb.at[idx, 'fc26_pos'] = data['positions'] 
        players_fb.at[idx, 'fc26_foot'] = data['preferred_foot']

--- 3. Matching Players ---


In [53]:
# ==========================================
# 4. POSITION LOGIC & GROUPING
# ==========================================
print("--- 4. Determining Positions & Groups ---")

# Fill missing rows
cols_to_fill = ['fc26_pos', 'fc26_foot', 'fc26_roles']
players_fb[cols_to_fill] = players_fb.groupby('Url')[cols_to_fill].transform(lambda x: x.ffill().bfill())

def get_final_position(row):
    if pd.notna(row['fc26_pos']): return row['fc26_pos']
    
    # Fallback Logic
    fb_pos = row['Pos']
    foot = row['fc26_foot'] 
    
    if fb_pos in ['DF', 'DF,FW', 'FW,DF']:
        if foot == 'Left': return 'LB'
        elif foot == 'Right': return 'RB'
        else: return 'CB'
    if fb_pos in ['MF', 'MF,DF', 'DF,MF']: return 'CDM'
    if fb_pos == 'MF,FW': return 'CAM'
    if fb_pos == 'FW': return 'ST'
    if fb_pos == 'FW,MF': return 'RW' 
    if fb_pos == 'GK': return 'GK'
    return fb_pos

players_fb['final_pos'] = players_fb.apply(get_final_position, axis=1)

# --- Assign Position Groups ---
def get_pos_group(pos):
    pos = str(pos)
    if 'GK' in pos: return 'GK'
    if any(x in pos for x in ['CB', 'LB', 'RB', 'LWB', 'RWB', 'DF']): return 'DEF'
    if any(x in pos for x in ['CDM', 'CM', 'CAM', 'LM', 'RM', 'MF']): return 'MID'
    return 'FWD' # ST, CF, LW, RW

players_fb['pos_group'] = players_fb['final_pos'].apply(get_pos_group)

# Clean Role Strings
players_fb['roles'] = (
    players_fb['fc26_roles'].astype(str)
    .replace('nan', '')
    .str.replace('+', '') # Remove any lingering plus signs
    .str.replace(r'\s*,\s*', ', ', regex=True)
    .str.strip(', ')
)

--- 4. Determining Positions & Groups ---


C:\Users\ASUS\AppData\Local\Temp\ipykernel_25940\1456903923.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  players_fb[cols_to_fill] = players_fb.groupby('Url')[cols_to_fill].transform(lambda x: x.ffill().bfill())


In [54]:
# ==========================================
# 5. POSITION-SPECIFIC IMPUTATION 
# ==========================================
print("--- 5. Smart AI Imputation (Position Locked) ---")

stat_features = [
    'Goals', 'Assists', 'xG', 'npxG', 'xAG', 
    'PrgC', 'PrgP', 'Tkl', 'Int', 'Blocks', 
    'SCA', 'GCA', 'Cmp', 'Att', 'Succ', 'AerWon'
]
available_stats = [col for col in stat_features if col in players_fb.columns]

# We loop through groups to ensure GKs only match GKs, DEFs match DEFs, etc.
groups = ['GK', 'DEF', 'MID', 'FWD']

for group in groups:
    # Filter for this specific group
    group_mask = players_fb['pos_group'] == group
    group_df = players_fb[group_mask].copy()
    
    # Within this group, separate Known vs Unknown roles
    train_mask = (group_df['roles'].notna()) & (group_df['roles'] != '') & (group_df['roles'] != 'None')
    predict_mask = ~train_mask
    
    train_subset = group_df[train_mask]
    predict_subset = group_df[predict_mask]
    
    if len(train_subset) > 0 and len(predict_subset) > 0:
        # Fill NaN stats
        train_subset[available_stats] = train_subset[available_stats].fillna(0)
        predict_subset[available_stats] = predict_subset[available_stats].fillna(0)
        
        # Scale
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_subset[available_stats])
        X_predict = scaler.transform(predict_subset[available_stats])
        
        # Train KNN 
        knn = KNeighborsClassifier(n_neighbors=1, metric='cosine')
        knn.fit(X_train, train_subset['roles'])
        
        # Predict
        predicted_roles = knn.predict(X_predict)
        
        # Update Main DataFrame
        prediction_map = dict(zip(predict_subset.index, predicted_roles))
        for idx, role in prediction_map.items():
            players_fb.at[idx, 'roles'] = f"{role} (Est)"
            
        print(f"✅ Group {group}: Filled {len(prediction_map)} missing players.")
    else:
        print(f"⚠️ Group {group}: No imputation needed or not enough training data.")

--- 5. Smart AI Imputation (Position Locked) ---
✅ Group GK: Filled 573 missing players.
✅ Group DEF: Filled 2108 missing players.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_25940\3714875377.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_subset[available_stats] = train_subset[available_stats].fillna(0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25940\3714875377.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_subset[available_stats] = predict_subset[available_stats].fillna(0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25940\3714875377.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

✅ Group MID: Filled 2112 missing players.
✅ Group FWD: Filled 842 missing players.


In [55]:
# ==========================================
# 6. FINAL CLEANUP & SPLIT
# ==========================================
print("--- 6. Cleanup & Splitting ---")

# Split Position and Roles
def split_feature_column(df, source_col, prefix_name):
    cleaned_col = df[source_col].astype(str).replace('nan', '').str.replace(r'\(Est\)', '', regex=True)
    cleaned_col = cleaned_col.str.replace(r'\s*,\s*', ', ', regex=True).str.strip(', ')
    
    non_empty = cleaned_col[cleaned_col != '']
    if non_empty.empty: return df

    max_items = int(non_empty.str.count(',').max() + 1)
    
    if prefix_name == 'pos':
        new_cols = ['main_pos'] + [f'secondary_pos_{i}' for i in range(1, max_items)]
    else:
        new_cols = [f'{prefix_name}_{i}' for i in range(1, max_items + 1)]
        
    split_data = cleaned_col.str.split(', ', expand=True)
    df[new_cols[:split_data.shape[1]]] = split_data
    return df

players_fb = split_feature_column(players_fb, 'final_pos', 'pos')
players_fb = split_feature_column(players_fb, 'roles', 'role')

# Drop temporary columns
cols_to_drop = ['clean_name', 'clean_club', 'is_active', 'fc26_pos', 'fc26_foot', 'fc26_roles', 'pos_group']
players_fb = players_fb.drop(columns=cols_to_drop, errors='ignore')

players_fb = players_fb.rename(columns={'final_pos': 'Positions', 'roles': 'Roles', 'fc26_foot': 'Preferred foot'})

print("\n--- Process Complete ---")
print(players_fb[['Player', 'Positions', 'Roles']].head(10))

--- 6. Cleanup & Splitting ---

--- Process Complete ---
                  Player Positions                                     Roles
0        aaron anselmino        CB     Defender, Ball-playing defender (Est)
1  aaron ciammaglichella       CDM  Box-to-Box, Playmaker, Box crasher (Est)
2  aaron ciammaglichella       CDM  Box-to-Box, Playmaker, Box crasher (Est)
3  aaron ciammaglichella       CDM  Box-to-Box, Playmaker, Box crasher (Est)
4        aaron cresswell     CB,LB                        Fullback, Defender
5        aaron cresswell     CB,LB                        Fullback, Defender
6        aaron cresswell     CB,LB                        Fullback, Defender
7        aaron cresswell     CB,LB                        Fullback, Defender
8        aaron cresswell     CB,LB                        Fullback, Defender
9           aaron hickey        RB                 Fullback, Wide back (Est)


In [56]:
# ==========================================
# 7. FINAL REPORT
# ==========================================
print("\n--- Processing Complete ---")
print("Total Players:", len(players_fb))
print("Final Columns:", players_fb.columns.tolist())

print("\n--- Sample Data ---")
cols_to_show = ['Player', 'Positions', 'Roles', 'role_1']
if 'role_2' in players_fb.columns: cols_to_show.append('role_2')
print(players_fb[cols_to_show].head(10))


--- Processing Complete ---
Total Players: 10476
Final Columns: ['Player', 'Team', 'Season_End_Year', 'Comp', 'Nation', 'Pos', 'Age', 'Born', 'Url', 'Cmp_Total', 'Att_Total', 'TotDist_Total', 'PrgDist_Total', 'Cmp_Short', 'Att_Short', 'Cmp_Medium', 'Att_Medium', 'Cmp_Long', 'Att_Long', 'Ast_Standard', 'xAG', 'xA_Expected', 'A_minus_xAG_Expected', 'KP', 'Final_Third', 'PPA', 'CrsPA', 'PrgP', 'Cmp_percent_Total', 'Cmp_percent_Short', 'Cmp_percent_Medium', 'Cmp_percent_Long', 'MP_Playing.Time', 'Min_Playing.Time', 'Starts_Starts', 'Compl_Starts', 'Subs_Subs', 'unSub_Subs', 'PPM_Team.Success', 'onG_Team.Success', 'onGA_Team.Success', 'On_minus_Off_Team.Success', 'onxG_Team.Success..xG.', 'onxGA_Team.Success..xG', 'On_minus_Off_Team.Success..xG', 'Mn_per_MP_Playing.Time', 'Min_percent_Playing.Time', 'Mn_per_Start_Starts', 'Mn_per_Sub_Subs', 'plus_per__minus__Team.Success', 'plus_per__minus_90_Team.Success', 'xGplus_per__minus__Team.Success..xG', 'xGplus_per__minus_90_Team.Success..xG', 'To

In [57]:
players_fb

,Player,Team,Season_End_Year,Comp,Nation,Pos,Age,Born,Url,Cmp_Total,...,CrdR,Positions,Roles,main_pos,secondary_pos_1,secondary_pos_2,role_1,role_2,role_3,role_4
0,aaron anselmino,chelsea,2025,ENG-Premier League,ARG,DF,19.0,2005.0,https://fbref.com/en/players/1d87670e/Aaron-An...,0.0,...,0.0,CB,"Defender, Ball-playing defender (Est)",CB,None,None,Defender,Ball-playing defender,None,None
1,aaron ciammaglichella,torino,2023,Serie A,ITA,MF,17.0,2005.0,https://fbref.com/en/players/20525799/Aaron-Ci...,0.0,...,0.0,CDM,"Box-to-Box, Playmaker, Box crasher (Est)",CDM,None,None,Box-to-Box,Playmaker,Box crasher,None
2,aaron ciammaglichella,torino,2024,Serie A,ITA,MF,18.0,2005.0,https://fbref.com/en/players/20525799/Aaron-Ci...,0.0,...,0.0,CDM,"Box-to-Box, Playmaker, Box crasher (Est)",CDM,None,None,Box-to-Box,Playmaker,Box crasher,None
3,aaron ciammaglichella,torino,2025,ITA-Serie A,ITA,MF,19.0,2005.0,https://fbref.com/en/players/20525799/Aaron-Ci...,0.0,...,0.0,CDM,"Box-to-Box, Playmaker, Box crasher (Est)",CDM,None,None,Box-to-Box,Playmaker,Box crasher,None
4,aaron cresswell,west ham,2021,Premier League,ENG,DF,30.0,1989.0,https://fbref.com/en/players/4f974391/Aaron-Cr...,1586.0,...,0.0,"CB,LB","Fullback, Defender",CB,LB,None,Fullback,Defender,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,zion suzuki,parma,2025,ITA-Serie A,JPN,GK,21.0,2002.0,https://fbref.com/en/players/51e1f4f5/Zion-Suzuki,1008.0,...,1.0,GK,Goalkeeper (Est),GK,None,None,Goalkeeper,None,None,None
10472,zito luvumbo,cagliari,2022,Serie A,ANG,"FW,MF",19.0,2002.0,https://fbref.com/en/players/b09e0873/Zito-Luv...,0.0,...,0.0,"LM,ST,RM","False 9, Poacher, Inside forward, Inside forward",LM,ST,RM,False 9,Poacher,Inside forward,Inside forward
10473,zito luvumbo,cagliari,2024,Serie A,ANG,"FW,MF",21.0,2002.0,https://fbref.com/en/players/b09e0873/Zito-Luv...,246.0,...,0.0,"LM,ST,RM","False 9, Poacher, Inside forward, Inside forward",LM,ST,RM,False 9,Poacher,Inside forward,Inside forward
10474,zito luvumbo,cagliari,2025,ITA-Serie A,ANG,"FW,MF",22.0,2002.0,https://fbref.com/en/players/b09e0873/Zito-Luv...,181.0,...,0.0,"LM,ST,RM","False 9, Poacher, Inside forward, Inside forward",LM,ST,RM,False 9,Poacher,Inside forward,Inside forward


In [58]:
import os

desktop = os.path.join(os.path.expanduser("~"), "Desktop")

# Save player dataset
players_fb.to_csv(os.path.join(desktop, "players_integration.csv"), index=False)

print(f"✅ Files saved to: {desktop}")

✅ Files saved to: C:\Users\ASUS\Desktop
